In [1]:
# Importing libraries
import pandas as pd
import plotly.graph_objects as go
import dash
import dash_bootstrap_components as dbc
from dash import dcc, html
from dash.dependencies import Input, Output

# Sample data for demonstration
agri_fdi = pd.read_csv('FAOSTAT_data_en_4-26-2024.csv', encoding="ISO-8859-1")

# Extract unique values for dropdown options
country_list = agri_fdi["Area"].unique().tolist()
element_list = agri_fdi["Element"].unique().tolist()
item_list = agri_fdi["Item"].unique().tolist()
year_list = agri_fdi["Year"].unique().tolist()

# Create a dash application
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.UNITED])

# Create an app layout
app.layout = html.Div(children=[
    html.H1("Agriculture FDI Dashboard",
            style={'textAlign': 'center', 'color': '#503D36', 'font-size': 40}),
    dcc.Dropdown(id='countries', options=[{'label': i, 'value': i} for i in country_list],
                 value=None, style={"width": '60%'}, placeholder="Countries",
                 searchable=True),
    html.Br(),

    dcc.Dropdown(id='fdi-item', options=[{'label': i, 'value': i} for i in item_list],
                 value=None, style={"align": "center", "width": '60%', 'color': '#503D36'},
                 placeholder="FDI Item", searchable=True),
    html.Br(),

    dcc.Dropdown(id='fdi-element', options=[{'label': i, 'value': i} for i in element_list],
                 value=None, style={"align": "center", "width": '60%', 'color': '#503D36'},
                 placeholder="FDI Element", searchable=True),
    html.Br(),

       

    html.Div(dcc.Graph(id='bar-chart')),
    html.Br()
])


@app.callback(
    Output(component_id='fdi-item', component_property='options'),
    Input(component_id='countries', component_property='value')
)
def get_fdi_item(country_name):
    if country_name is not None:
        agri_fdi_filtered = agri_fdi[agri_fdi["Area"] == country_name]
        return [{'label': i, 'value': i} for i in agri_fdi_filtered['Item'].unique().tolist()]
    else:
        return []


@app.callback(
    Output(component_id='fdi-element', component_property='options'),
    [Input(component_id='countries', component_property='value'),
     Input(component_id='fdi-item', component_property='value')]
)
def get_fdi_element(country_name, item_name):
    if country_name is not None and item_name is not None:
        agri_fdi_filtered = agri_fdi[(agri_fdi["Area"] == country_name) & (agri_fdi["Item"] == item_name)]
        return [{'label': i, 'value': i} for i in agri_fdi_filtered['Element'].unique().tolist()]
    else:
        return []


@app.callback(
    Output(component_id='bar-chart', component_property='figure'),
    [Input(component_id='countries', component_property='value'),
     Input(component_id='fdi-item', component_property='value'),
     Input(component_id='fdi-element', component_property='value')]
)
def update_bar_chart(country_name, item_name, element_name):
    if country_name is not None and item_name is not None and element_name is not None:
        agri_fdi_filtered = agri_fdi[(agri_fdi["Area"] == country_name) & (agri_fdi["Item"] == item_name) &
                                      (agri_fdi["Element"] == element_name)]

        # Create a Plotly figure
        fig = go.Figure()

        # Add traces
        fig.add_trace(go.Bar(x=agri_fdi_filtered['Year'], y=agri_fdi_filtered['Value'],
                             marker=dict(color="Orange"),
                             text=agri_fdi_filtered['Value'],
                             textfont=dict(color='white', size=14, family='Arial'),
                             textposition="inside", name=element_name))

        fig.update_layout(barmode="group",
                          title_text=f'Year-Wise Analysis of FDI in Agriculture Sector - {item_name}',
                          margin=dict(t=50, l=25, r=25, b=100),
                          xaxis_tickfont_size=11,
                          yaxis=dict(title='Value', titlefont_size=14, tickfont_size=12),
                          coloraxis=dict(colorscale=['blue', 'green']), showlegend=True,
                          paper_bgcolor="White",
                          plot_bgcolor="White",
                          )

        return fig
    else:
        return {}


if __name__ == '__main__':
    app.run_server(debug=True, port=8080, mode='external')